# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation (RELU JE X NA R+, 0 NA R-)
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
]) # TEDY JEN INPUT X (BERE SE JAKO JEDEN INPUT), JEDEN LAYER S 10 NEURONAMA A OUTPUT CISLO

model.summary()

# MYSLIM ZE 90 PARAMETRU V HIDDEN PROTOZE 10*8 (10 NEURONU A X MA 8 SLOUPCU) + 10 BIAS PARAMETRU. U OUTPUT PAK 10 Z NEURONU A 1 BIAS

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Specification B) - TOTO NEDELAL, ALTERNATIVNI ZPUSOB JAK TO UDELAT

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [8]:
inputs = tf.keras.Input(shape=(X_train.shape[1],)) # V SEKVENCNIM NAPSAL TY LAYERS JAK JSOU ZA SEBOU, TADY ZADA CO TAM MA BYT A TEN MODEL SE NAVRHNE SAM

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [9]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [10]:
# TODO: To make the following line work you need to install graphviz (if you have not done so in one of the previous classes)
# 1) follow the instructions https://graphviz.gitlab.io/download/?fbclid=IwAR1V-lrRhho5rSfBVYXYISsighqRwOCOgMHLmL_DclkQrPtMXQaKj3mFcqs
# 2) this notebook has been tested with version 8.0.3
# 3) make sure you add it to the PATH variable (you are specifically asked during the installation) at least for local user

tf.keras.utils.plot_model(model) # ASI PRCAM, neni to nejakej zazrak grafickej

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Model compilation and training

In [12]:
# ZATIM Jsem zkonstruoval tu sit jakoby, ale porad mi chybi prpojeni s outputy, potrebuju LOSS funkci abych vyhodnotil jak daleko je nas output 
# od realneho. Pouzijeme binary cross entropy, binary referencuje typ outputu, cross entropy se pouziva kdyz je output pravdepodobnostni rozdeleni 
# a chceme merit jako daleko je prave Y od predikovaneho. C-E se pocita jako -E_{Y_true}(log(Y_pred)). Y_true je brano jako nahodne.
# Realne teda pocitame -sum_{k=1}^N y_true^(k) * log(y_pred^(k))

In [13]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [14]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.04433858]
 [-0.48884207]
 [-0.14795089]
 [ 0.49902743]
 [-0.43155372]
 [-0.34787712]
 [-0.45331258]
 [ 0.21712661]
 [-0.5472337 ]
 [ 0.41418248]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-0.13606709  0.28868985  0.20628262 -0.38618106  0.3443514   0.4406668
  -0.4625658   0.3101338   0.02230495 -0.34151125]
 [ 0.34663117  0.47178292 -0.5677739   0.15289944 -0.17409372 -0.5652492
   0.10102141 -0.33703676  0.31804562 -0.38921916]
 [-0.10551062 -0.14723098 -0.4651388   0.46210492 -0.44629174 -0.24044785
   0.09708405  0.4910314   0.5610268  -0.08296752]
 [ 0.00611526 -0.22575524 -0.17058706 -0.31471995 -0.551751   -0.41084895
   0.01788127  0.23742157  0.42155093  0.13149142]
 [ 0.09500605 -0.4300148  -0.45128986 -0.5492053  -0.3490768   0.57227325
   0.20611864 -0.4884621  -0.4377318  -0.06032985]
 [ 0.38897717 -0.5671009  -0.07463712 -0.20924884 -0.31475216 -

In [15]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100) # BATCH SIZE JE KOLIK INPUTU POUZIJU NA VYPOCET LOSS, TYPICKY HODNOTA 64, MENE NE, MAX TREBA 2000

Epoch 1/100
200/200 [==============================] - 1s 2ms/step - loss: 0.7509 - auc: 0.5350 - binary_accuracy: 0.5270
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 0.5519 - auc: 0.8141 - binary_accuracy: 0.7460
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 0.4330 - auc: 0.9034 - binary_accuracy: 0.8234
Epoch 4/100
200/200 [==============================] - 0s 997us/step - loss: 0.3682 - auc: 0.9267 - binary_accuracy: 0.8537
Epoch 5/100
200/200 [==============================] - 0s 985us/step - loss: 0.3311 - auc: 0.9388 - binary_accuracy: 0.8711
Epoch 6/100
200/200 [==============================] - 0s 963us/step - loss: 0.3076 - auc: 0.9459 - binary_accuracy: 0.8817
Epoch 7/100
200/200 [==============================] - 0s 964us/step - loss: 0.2926 - auc: 0.9502 - binary_accuracy: 0.8880
Epoch 8/100
200/200 [==============================] - 0s 947us/step - loss: 0.2830 - auc: 0.9530 - binary_accuracy: 0.8909
Epoch 9/100
20

In [16]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 0s 2ms/step - loss: 0.1907 - auc: 0.9795 - binary_accuracy: 0.9423


In [17]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.36
0 - 0.33
0 - 0.34
1 - 1.00
0 - 0.00
0 - 0.04
1 - 0.97
0 - 0.01
1 - 0.97
1 - 1.00


### Add early stopping and regularization

In [24]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs) # l2 REGULARIZACE
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_8 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu_3 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid_3 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [25]:
epochs = 500

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/500
200/200 [==============================] - 1s 2ms/step - loss: 0.6542 - auc_3: 0.7018 - binary_accuracy: 0.6420 - val_loss: 0.5757 - val_auc_3: 0.8194 - val_binary_accuracy: 0.7344
Epoch 2/500
200/200 [==============================] - 0s 1ms/step - loss: 0.5190 - auc_3: 0.8551 - binary_accuracy: 0.7692 - val_loss: 0.4559 - val_auc_3: 0.8934 - val_binary_accuracy: 0.8050
Epoch 3/500
200/200 [==============================] - 0s 1ms/step - loss: 0.4324 - auc_3: 0.9005 - binary_accuracy: 0.8163 - val_loss: 0.3917 - val_auc_3: 0.9197 - val_binary_accuracy: 0.8425
Epoch 4/500
200/200 [==============================] - 0s 1ms/step - loss: 0.3841 - auc_3: 0.9215 - binary_accuracy: 0.8429 - val_loss: 0.3570 - val_auc_3: 0.9332 - val_binary_accuracy: 0.8584
Epoch 5/500
200/200 [==============================] - 0s 1ms/step - loss: 0.3568 - auc_3: 0.9322 - binary_accuracy: 0.8577 - val_loss: 0.3397 - val_auc_3: 0.9391 - val_binary_accuracy: 0.8681
Epoch 6/500
200/200 [==============

### Gridsearch and Tensorboard - JAK TEDA NAJIT SPRAVNE HYPERMARAMETRY?
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

PRO RUZNE KOMBINACE NATRENUJE MODELY S NIZSIM POCTEM EPOCH A PAK SROVNA VYKONNOST POMOCI TENSORFLOW UI

In [33]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 30
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/30
200/200 [==============================] - 1s 5ms/step - loss: 0.7762 - AUC: 0.6801 - binary_accuracy: 0.6295 - val_loss: 0.6788 - val_AUC: 0.7462 - val_binary_accuracy: 0.6591
Epoch 2/30
200/200 [==============================] - 0s 1ms/step - loss: 0.6243 - AUC: 0.8022 - binary_accuracy: 0.6895 - val_loss: 0.5844 - val_AUC: 0.8442 - val_binary_accuracy: 0.7225
Epoch 3/30
200/200 [==============================] - 0s 1ms/step - loss: 0.5624 - AUC: 0.8562 - binary_accuracy: 0.7573 - val_loss: 0.5412 - val_AUC: 0.8754 - val_binary_accuracy: 0.7897
Epoch 4/30
200/200 [==============================] - 0s 1ms/step - loss: 0.5332 - AUC: 0.8738 - binary_accuracy: 0.7911 - val_loss: 0.5190 - val_AUC: 0.8867 - val_binary_accuracy: 0.8081
Epoch 5/30
200/200 [==============================] - 0s 1ms/step - loss: 0.5172 - AUC: 0.8791 - binary_accuracy: 0.7945 - val_loss: 0.5057 - val_AUC: 0.8906 - val_binary_accuracy: 0.8128
Epoch 6/30
200/200 [==============================] - 0s 1ms

In [34]:
%load_ext tensorboard
%tensorboard --logdir=logs/binary_classification_test

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 26528), started 0:03:36 ago. (Use '!kill 26528' to kill it.)